In [12]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold

#1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" 
#и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных 
#может занять несколько минут

newsgroups = datasets.fetch_20newsgroups(subset = 'all', categories=['alt.atheism', 'sci.space'])
x = newsgroups.data
y = newsgroups.target

#2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам 
#вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве 
#используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не
#используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда 
#признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться
#при обучении алгоритма.

vectorizer = TfidfVectorizer()
vectorizer.fit_transform(x)

#3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM 
#с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр
#random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

grid = {'C': np.power(10.0, np.arange(-5,6))}
cv = KFold(y.size, n_folds = 5, random_state = 241, shuffle = True)
clf = SVC(kernel = 'linear', random_state = 241)
gs = GridSearchCV(clf, grid, scoring = 'accuracy', cv = cv)
gs.fit(vectorizer.transform(x),y)

C = gs.best_params_.get('C')

#4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

clf = SVC(C = C, kernel='linear', random_state=241)
clf.fit(vectorizer.transform(x),y)

#5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). 
#Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре,
#в лексикографическом порядке.

idxs = np.argsort(-np.abs(clf.coef_.todense().data))
feature_maping = vectorizer.get_feature_names()
print(' '.join(sorted([feature_maping[i] for i in idxs[0][0:10]])))


atheism atheists bible god keith moon religion sci sky space
